# Определение токсичных комментариев


Имеется датасет с текстами-комментариями, в котором есть разметка, является ли комментарий токсичным.

Моя задача заключается в том, чтобы на основе предоставленных данных обучить модель, которая в других текстах-комментариях будет искать токсичные комментарии (т.е.классифицировать комментарии). 

Я выполню задачу сначала с использование TFIDF в качестве признаков, а затем буду использовать BERT.

Оценивать модели буду метрикой F1.

Проведу декомпозицию задачи:
1. Импортирую инструменты и проведу обзор данных.
2. Использование для обучения TFIDF.

 Лемматизация, очистка.

  Подгтовка признаков, разбиение на выборки.

  Обучение двух моделей, проверка по метрике F1_score.

3. Использование для обучения BERT.

## Импорт библиотек и обзор данных

In [ ]:
!pip install torch
!pip install transformers
!pip install catboost
!pip install spacy

In [ ]:
import numpy as np
import pandas as pd
from tqdm import notebook

import torch
import transformers
import spacy

import re

import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from transformers import AutoModel, AutoTokenizer

from catboost import CatBoostRegressor, CatBoostClassifier, cv, Pool
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score, f1_score, precision_recall_curve, precision_score, recall_score, roc_auc_score
from sklearn.model_selection import cross_val_score, GridSearchCV, RandomizedSearchCV, train_test_split


In [ ]:
nlp = spacy.load("en_core_web_sm") #сразу импортирую модель для лемматизации с английским языком

In [ ]:
import os

pth1 = '/content/toxic_comments.csv'
pth2 = '/datasets/toxic_comments.csv'

if os.path.exists(pth1):
  data = pd.read_csv(pth1)
elif os.path.exists(pth2):
  data = pd.read_csv(pth2)
else:
  print('Something is wrong')

Возьму для TFIDF и для BERT весь датасет

In [ ]:
data

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
...,...,...
159566,""":::::And for the second time of asking, when ...",0
159567,You should be ashamed of yourself \n\nThat is ...,0
159568,"Spitzer \n\nUmm, theres no actual article for ...",0
159569,And it looks like it was actually you who put ...,0


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [ ]:
data.isna().sum()

text     0
toxic    0
dtype: int64

In [ ]:
data.duplicated().sum()

0

Столбец text для подготовки признаков, toxic - целевой признак. Комментарии на английском языке.

In [ ]:
data_bert = pd.read_csv('/content/toxic_comments.csv')
data_bert = data.sample(150000).reset_index(drop=True) # Выделю в отдельную переменную данные для BERT (150000 текстов)

## TFIDF

### Лемматизация и очистка регуляризацией (SpaCy)

Проведу лемматизацию с помощью SpaCY

In [ ]:
%%time
data['lemm_text'] = data['text'].apply(lambda row: " ".join([w.lemma_ for w in nlp(row)]))

CPU times: user 35min 30s, sys: 12.9 s, total: 35min 43s
Wall time: 36min 6s


Теперь с помощью регуляризации избавлюсь от лишних слов и символов:

In [ ]:
def clear_text(text):
    sub = re.sub(r'[^a-zA-Z]', ' ', text).split()
    return ' '.join(sub)

In [ ]:
data['lemm_text'] = data['lemm_text'].apply(clear_text)

Теперь подготовлю список стопслов, чтобы использовать в дальнейшем для TFIDF:

In [ ]:
nltk.download('stopwords')
stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


##### Доп. Лемматизация с помощью nltk с POS-тегами

Лемматизацию можно провести и другим способом, используя nltk c POS-тегами

In [ ]:
# from nltk.stem import WordNetLemmatizer
# from nltk.corpus import wordnet
# from nltk.tokenize import word_tokenize
# nltk.download('wordnet')
# nltk.download('averaged_perceptron_tagger')
# nltk.download('omw-1.4')



# def get_wordnet_pos(word):
#     tag = nltk.pos_tag([word])[0][1][0].upper()
#     tag_dict = {"J": wordnet.ADJ,
#                 "N": wordnet.NOUN,
#                 "V": wordnet.VERB,
#                 "R": wordnet.ADV}
#     return tag_dict.get(tag, wordnet.NOUN)

# lemmatizer = WordNetLemmatizer()

# lemm_texts_list = []
# for text in data['text']:
#   word_list = nltk.word_tokenize(text)
#   for word in word_list:
#     word_lem = lemmatizer.lemmatize(word, get_wordnet_pos(word))
#     lemm_texts_list.append(word_lem)

[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package omw-1.4 to /home/jovyan/nltk_data...


In [ ]:
#lemm_texts_list

### Подготовка признаков

В качестве новых признаков для обучения будет TFIDF на подготовленных данных. Разделю выборки:

In [ ]:
features = data['lemm_text']
target = data['toxic']
features_train, features_test, target_train, target_test = train_test_split(features, target, random_state=123, test_size=0.2)

В качестве новых признаков для обучения будет TFIDF на подготовленных данных, поэтому создам объект TfidfVectorizer с передачей стоплслов:

In [ ]:
tf_idf = TfidfVectorizer(stop_words=stop_words)

Теперь создам TFIDF для тренировочной и для тестовой выборок:

In [ ]:
tf_idf_train = tf_idf.fit_transform(features_train)
tf_idf_test = tf_idf.transform(features_test)

### Обучение моделей

#### Логистическая регрессия

Проверю сначала на более простой модели, передам модели созданные признаки и таргет, показывающий, токсичный ли комментарий. Метрика для оценки качества - F1.

In [ ]:
param = {'C': [100000000, 1000000000, 100000000]}

In [ ]:
%%time
log_model = LogisticRegression(random_state=123, n_jobs=-1)
random_search = RandomizedSearchCV(log_model, param_distributions=param, cv=3, random_state=123, scoring='f1')
random_search.fit(tf_idf_train, target_train)

print(random_search.best_estimator_ , 
      random_search.best_params_, 
      random_search.best_score_)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:296: UserWarning: The total space of parameters 3 is smaller than n_iter=10. Running 3 iterations. For exhaustive searches, use GridSearchCV.
  UserWarning,


LogisticRegression(C=1000000000, n_jobs=-1, random_state=123) {'C': 1000000000} 0.7372006492911082
CPU times: user 770 ms, sys: 365 ms, total: 1.14 s
Wall time: 44.5 s


In [ ]:
log_model = LogisticRegression(random_state=123, n_jobs=-1)
cross_val_score(log_model, tf_idf_train, target_train, scoring='f1', cv=3)

array([0.71463103, 0.71069182, 0.7055794 ])

Оценка на кроссвалидации 0.737, без настройки регуляризации "С" метрика ниже.

#### Catboost

In [ ]:
CB_train_data = Pool(data=tf_idf_train,
                  label=target_train)

In [ ]:
CB_test_data = Pool(data=tf_idf_test,
                  label=target_test)

In [ ]:
%%time
CB_model = CatBoostClassifier(loss_function='Logloss', eval_metric='F1', early_stopping_rounds=120, task_type='GPU')
CB_model.fit(CB_train_data)

In [ ]:
%%time
cb_test_predict = CB_model.predict(tf_idf_test)

CPU times: user 1.21 s, sys: 67 ms, total: 1.28 s
Wall time: 1.12 s


In [ ]:
print('F1_score на тестовой выброке для Catboost без подбора параметров:', f1_score(target_test, cb_test_predict))

F1_score на тестовой выброке для Catboost без подбора параметров: 0.7697068996905152


Без подбора гиперпараметров Catboost показывает метрику F1 = 0.769. 
Время обучения: 36 мин.

*Можно также использовать подбор гиперпараметров, но как оказалось далее, с подходящей моделью BERT Catboost показывает метрику F1 намного выше.*

In [ ]:
# %%time
# CB_model = CatBoostClassifier(loss_function='Logloss', eval_metric='F1', early_stopping_rounds=120)

# params = {'learning_rate': [0.05, 0.1],
#            'depth': [4, 6]}

# grid_result = CB_model.grid_search(params, CB_train_data, plot=True, cv=3, partition_random_seed=123, verbose=False)

In [ ]:
# print('Лучшие параметры:', grid_result.params)
# grid_result.cv_results

## BERT

С использованием BERT метрика F1 на наших данных должна стать выше. Я нашел модель toxic-bert, которая должна подойти для решения задачи.

In [ ]:
model_name = "unitary/toxic-bert"

### Токенизация и ограничение количества токенов


С помощью скачанного словаря инициализирую класс для токенизации, проведу токенизацию с добавлением токенов начала и конца.

Также для корректного создания эмбеддингов ограничю количество токенов в каждом тексте (model_max_length=512).

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, model_max_length=512)

tokenized = data_bert['text'].apply(
    lambda x: tokenizer.encode(x, add_special_tokens=True, truncation=True))

### Padding

Теперь сделаю длину текстов равными, вычислив самый длинный текст и добавив нули к другим текстам. Далее помечу такие нули как неимеющие значимости для attention_mask:

In [ ]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])

attention_mask = np.where(padded != 0, 1, 0)

### Инициализация модели BERT

Теперь инициализирую саму модель по имени:

In [ ]:
model = AutoModel.from_pretrained(model_name)

Some weights of the model checkpoint at unitary/toxic-bert were not used when initializing BertModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


### Создание эмбеддингов

Создавать эмбеддинги будем по батчам, на каждом батче модель будет получать данные в виде тензоров и маску. Таким образом список эмбеддингов будет пополняться по каждому батчу с данными

In [ ]:
%%time
batch_size = 120
embeddings = []
for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
        batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]).cuda()
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)]).cuda()
        
        print(len(batch))
        with torch.no_grad():
          model.cuda()
          batch_embeddings = model(batch, attention_mask=attention_mask_batch)
        
        embeddings.append(batch_embeddings[0][:,0,:].cpu().numpy())
        
        del batch
        del attention_mask_batch
        del batch_embeddings

  0%|          | 0/8 [00:00<?, ?it/s]

120
120
120
120
120
120
120
120
CPU times: user 28.2 s, sys: 59 ms, total: 28.3 s
Wall time: 30.1 s


### Подготовка признаков

Теперь у меня есть эмбеддинги-признаки, объединю их в одну матрицу признаков, далее подготовлю все выборки для обучения моделей:


In [ ]:
embeddings

In [ ]:
features_bert = np.concatenate(embeddings)
target_bert = data_bert['toxic']

features_train_bert, features_test_bert, target_train_bert, target_test_bert = train_test_split(features_bert, target_bert, random_state = 123, test_size=.5)

### Обучение моделей

### Логистическая регрессия

In [ ]:
logreg_bert = LogisticRegression(random_state=123)
logreg_bert.fit(features_train_bert, target_train_bert)

pred_bert = logreg_bert.predict(features_test_bert)
print(f1_score(target_test_bert, pred_bert))

LogisticRegression(random_state=123)

#### Catboost

In [ ]:
CB_train_data_bert = Pool(data=features_train_bert,
                  label=target_train_bert)

CB_test_data_bert = Pool(data=features_test_bert,
                  label=target_test_bert)

In [ ]:
%%time
CB_model_bert = CatBoostClassifier(loss_function='Logloss', eval_metric='F1', early_stopping_rounds=120, auto_class_weights='Balanced', task_type='GPU')
CB_model_bert.fit(CB_train_data_bert)

Learning rate set to 0.035021
0:	learn: 0.9541284	total: 43.8ms	remaining: 43.8s
1:	learn: 0.9904762	total: 141ms	remaining: 1m 10s
2:	learn: 0.9807692	total: 230ms	remaining: 1m 16s
3:	learn: 0.9904762	total: 322ms	remaining: 1m 20s
4:	learn: 1.0000000	total: 379ms	remaining: 1m 15s
5:	learn: 1.0000000	total: 441ms	remaining: 1m 13s
6:	learn: 1.0000000	total: 477ms	remaining: 1m 7s
7:	learn: 1.0000000	total: 533ms	remaining: 1m 6s
8:	learn: 1.0000000	total: 587ms	remaining: 1m 4s
9:	learn: 1.0000000	total: 636ms	remaining: 1m 2s
10:	learn: 0.9904762	total: 685ms	remaining: 1m 1s
11:	learn: 0.9904762	total: 737ms	remaining: 1m
12:	learn: 0.9904762	total: 786ms	remaining: 59.7s
13:	learn: 1.0000000	total: 838ms	remaining: 59s
14:	learn: 1.0000000	total: 893ms	remaining: 58.7s
15:	learn: 1.0000000	total: 951ms	remaining: 58.5s
16:	learn: 1.0000000	total: 999ms	remaining: 57.8s
17:	learn: 1.0000000	total: 1.05s	remaining: 57.4s
18:	learn: 1.0000000	total: 1.1s	remaining: 56.8s
19:	learn: 

In [ ]:
%%time
cb_test_predict_bert = CB_model_bert.predict(features_test_bert)

CPU times: user 23.1 ms, sys: 3.97 ms, total: 27.1 ms
Wall time: 23.6 ms


In [ ]:
print('F1_score на тестовой выброке для Catboost с BERT:', f1_score(target_test_bert, cb_test_predict_bert))

F1_score на тестовой выброке для Catboost с BERT: 0.9583333333333334


## Выводы

Для TFIDF при обучении с помощью Catboost на всем датафрейме удалось добиться требуемого показателя метрики (F1 = 0.769).

При использовании малой выборки для создания эмбеддингов и дальнейшего обучения модели, удалось получить метрику F1 > 0.9, думаю во многом это связано с хорошо подходящей моделью BERT.